In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/fake_train.csv


In [2]:
import pandas as pd
df=pd.read_csv("/kaggle/input/fake-news/fake_train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df=df.dropna()

In [4]:
X=df.drop('label',axis=1)
y=df['label']

In [5]:
X.shape

(18285, 4)

In [6]:
import tensorflow as tf
tf.__version__

'2.6.4'

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [8]:
messages=df.copy()
messages.reset_index(inplace=True)

In [9]:
import nltk
import re
from nltk.corpus import stopwords


In [10]:
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
voc_size=5000
one_hot_rep=[one_hot(words,voc_size) for words in corpus]
print(one_hot_rep[:5])

[[1961, 1790, 862, 3336, 4042, 1245, 4417, 2912, 3204, 2293], [3341, 3729, 353, 2365, 4789, 1064, 2188], [1490, 4139, 2589, 3662], [1366, 1137, 1428, 117, 4979, 2760], [93, 4789, 3164, 535, 1474, 3362, 4789, 4721, 3275, 1910]]


In [12]:
sent_len=20
embedded_docs=pad_sequences(one_hot_rep,padding='pre',maxlen=sent_len)
print(embedded_docs)
                    

[[   0    0    0 ... 2912 3204 2293]
 [   0    0    0 ... 4789 1064 2188]
 [   0    0    0 ... 4139 2589 3662]
 ...
 [   0    0    0 ... 4905 2465 1140]
 [   0    0    0 ... 2708 3329 1213]
 [   0    0    0 ...  246 1593 3037]]


In [13]:
dimensions=40
model=Sequential()
model.add(Embedding(voc_size,dimensions,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))


2022-11-21 07:35:40.974392: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [15]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [17]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


2022-11-21 07:35:41.502794: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


192/192 [==============================] - 9s 32ms/step - loss: 0.3339 - accuracy: 0.8428 - val_loss: 0.1922 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 5s 29ms/step - loss: 0.1377 - accuracy: 0.9451 - val_loss: 0.1909 - val_accuracy: 0.9215
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0949 - accuracy: 0.9637 - val_loss: 0.2274 - val_accuracy: 0.9200
Epoch 4/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0684 - accuracy: 0.9761 - val_loss: 0.2285 - val_accuracy: 0.9145
Epoch 5/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0429 - accuracy: 0.9862 - val_loss: 0.3043 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.4181 - val_accuracy: 0.9167
Epoch 7/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0165 - accuracy: 0.9949 - val_loss: 0.4409 - val_accuracy: 0.9125
Epoch 8/10